In [ ]:
#!pip install pandas requests beautifulsoup4

### Import modules

In [ ]:
import re
import os
import json
import shutil
import pandas
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

In [ ]:
strategies_csv = "strategies.csv"
if os.path.exists(strategies_csv):
    os.remove(strategies_csv)

strategies_path = "strategies"
if os.path.exists(strategies_path):
    shutil.rmtree(strategies_path)

### List public strategies

In [ ]:
resp = requests.request('GET', 'https://strat.ninja/strats.php')
strategies = []

if resp.status_code == 200:
    filtered_text = "\n".join([ line for line in resp.text.splitlines() if not re.search(r'private', line, re.IGNORECASE) ])
    strats = re.findall(r'href="overview.php\?strategy=(.*?)"', filtered_text)

    for strat in strats:
        if "action=open" not in strat:
            strategies.append(strat)

### Process function

In [ ]:
def process_strategy(strategy):
    resp = requests.request("GET", f"https://strat.ninja/overview.php?strategy={strategy}")

    # Get strategy tags
    soup = BeautifulSoup(resp.text, features="html.parser")
    tags = soup.find("div", class_="tags")
    metadata = {
        "strategy" : strategy,
        "mode" : None,
        "timeframe" : None,
        "failed" : False,
        "bias" : None,
        "stalled" : None,
        "leverage" : None,
        "profit" : 0,
        "can_short" : False,
    }

    # Clasify tags
    for tag in tags.find_all("a"):
        if not tag.find("img") and not tag.get("onclick"):
            text = tag.get_text()

            mode = [ "Spot", "Futures"]
            for element in mode:
                if element == text:
                    metadata["mode"] = text

            timeframes = [ "1m", "3m", "5m", "10m", "15m", "30m", "1h", "2h", "4h", "6h", "12h", "1d", "1w", ]
            for element in timeframes:
                if element == text:
                    metadata["timeframe"] = text

            failed = [ "Failed" ]
            for element in failed:
                if element == text:
                    metadata["failed"] = True

            bias = ["Biased (Lookahead Analysis)", "Bias unchecked", "Unbiased"]
            for element in bias:
                if element == text:
                    metadata["bias"] = text

            stalled = [ "Stalled - 90 Percent Negative", "Stalled - Biased", "Stalled - Negative", ]
            for element in stalled:
                if element == text:
                    metadata["stalled"] = text

            leverage = "X"
            if text.endswith(leverage):
                metadata["leverage"] = text
    
    profit = 0
    cum_prof = []
    
    # If strategy worked, then it must include a table of results. 
    # Sum profits.

    if 'Failed' not in resp.text:
        soup = BeautifulSoup(resp.text, features="html.parser")

        table = soup.find('table', id='example')
        if not table: return

        tbody = table.find('tbody')
        if not tbody: return

        rows = tbody.find_all('tr')
        for row in rows:
            columns = row.find_all('th')
            cum_prof.append(float(columns[5].text)) # Tot. Profit %

        if cum_prof:
            profit = sum(cum_prof) / len(cum_prof)

    metadata["profit"] = profit

    strategy_mirror = f'https://strat.ninja/mirror/{strategy}.py'
    resp = requests.request('GET', strategy_mirror)
    if resp.status_code == 200 and profit > 0:
        # Can Short?
        can_short = re.search(r'can_short\s*=\s*True', resp.text)
        if can_short:
            metadata["can_short"] = True

        # Strategy's indicators
        pattern = r'dataframe\["(.*?)"\]|dataframe\[\'(.*?)\'\]'
        matches = re.findall(pattern, resp.text)
        values = {m[0] or m[1] for m in matches}
        metadata["indicators"] = list(set(values))

        # Save code
        strategy_file = f"{strategies_path}/{strategy}.py"
        with open(strategy_file, "a+") as file:
            file.write(resp.text)

    # Dump to a file the results
    with open('strategies.csv', 'a+') as f:
        f.write(f'{ json.dumps(metadata["strategy"]) };{ json.dumps(metadata["mode"]) };{ json.dumps(metadata["timeframe"]) };{ json.dumps(metadata["failed"]) };{ json.dumps(metadata["bias"]) };{ json.dumps(metadata["stalled"]) };{ json.dumps(metadata["leverage"]) };{ json.dumps(metadata["profit"]) };{ json.dumps(metadata["can_short"]) };{ json.dumps(metadata["indicators"]) };\n')

    print(metadata)

### Process strategies in parallel

In [ ]:
with ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(process_strategy, strategies)

### Data Loading

In [ ]:
separator = ";"
filename  = "strategies.csv"
columns   = [ "strategy", "mode", "timeframe", "failed", "bias", "stalled", "leverage", "profit", "can_short", "indicators" ]

pandas.set_option('display.max_rows', 100)

df = pandas.read_csv(filename, sep=separator, names=columns)

### Data cleaning

In [ ]:
df = df[df["failed"] == False]

### Spot

In [ ]:
spots = df[df["mode"] == "Spot"]
spots = spots[spots["bias"] == "Unbiased"]
spots = spots[["strategy", "profit", "timeframe", "bias", "stalled", "can_short"]]
spots.sort_values("profit", ascending=False).head(25)

### Futures

In [ ]:
futures = df[df["mode"] == "Futures"]
futures = futures[futures["bias"] == "Unbiased"]
futures = futures[["strategy", "profit", "timeframe", "leverage", "bias", "stalled", "can_short"]]
futures.sort_values("profit", ascending=False).head(25)